# Determining an Average Meteorological Year

This notebook calculates an hourly time series for an average meteorological year (AMY) for a given location, representing the mean weather conditions for a one-year period. This was designed to provide a similar product to a [Typical Meteorological Year](https://nsrdb.nrel.gov/data-sets/tmy), an hourly dataset used for applications in energy and building systems modeling. Because this represents average rather than extreme conditions, an AMY dataset is not suited for designing systems to meet the worst-case conditions occurring at a location.

In the Cal-Adapt: Analytics Engine, there are three options for investigating AMY:  
1. Absolute AMY: represents the historical reference period (1981-2010) or a future reference period based around a 30-year warming level time window. **Note: Absolute AMY data and display is not bias corrected, and should be used for visual data inspection only.**
2. Difference AMY: represents the difference between a designated future 30-year period and the historical baseline (1981-2010)
3. *Coming soon!* Severe meteorological year: represents the difference between a designated extreme percentile for a designated future 30-year period and the historical baseline (1981-2010)

For any given location, we can calculate the desired AMY option and display one of the following three options depending on your selections:
1. Heatmap of AMY
2. Diurnal cycle of a typical day within the AMY
3. Annual cycle of a typical hour within the AMY

## Step 0: Setup

Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [ ]:
import panel as pn
pn.extension()

import climakitae as ck
import xarray as xr
import pandas as pd
import numpy as np
import datetime

import logging
logging.captureWarnings(capture=True)

Additionally, get set up to make the computing go faster by executing the following cell. It will likely take several minutes to spin up! Learn more about dask and see some common [troubleshooting tips on our FAQ page](https://analytics.cal-adapt.org/docs/faq/).

In [ ]:
from dask.distributed import progress
from dask_gateway import GatewayCluster
cluster = GatewayCluster()
cluster.adapt(minimum=0, maximum=16)
client = cluster.get_client()
cluster

To use *climakitae*, load a new application:

In [ ]:
app = ck.Application()

## Step 1: Explore the Average Meteorological Year

The below panel can be used to explore an AMY for a particular variable and geographic area of interest during the historical period, as well as comparisons to an AMY under future conditions at different global warming levels (for more information about warming levels, see the `explore_warming.ipynb` notebook). 

The absolute AMY for a given 30-year period (either the historical period 1981-2010, or a thirty year window centered on where each GCM simulation reaches the specified global warming level) is calculated by identifying for each hour the hourly value that is closest to the mean hourly value across all years to produce a full annual timeseries of hourly data for a variable of interest that best represents the average conditions.

An absolute AMY for the historical period can then be compared to an AMY under different warming levels, and the difference in hourly values of a given variable for an AMY under historical conditions and future conditions can be plotted using the "Difference" button.

The resulting heatmap of AMY values for a full annual timeseries is then plotted, with day of year 1 being January 1st and hour of day given in Pacific Standard Time. A static version of the plot can be saved by using the save button in the lower right of the plot window.

To learn more about the data available on the Analytics Engine, [see our data catalog](https://analytics.cal-adapt.org/data/). 

<span style="color:#FF0000">

**Note:** Once you have selected the options you would like to investigate, please select the "Reload Data" button. It will take 2-4 minutes to update, hang tight!

In [ ]:
%%time
app.explore.amy()

## Step 2: Application

Examining a particular month within the Average Meteorological Year can provide information towards a long-term energy yield analysis during a particular time of year. For example, a warm season month can provide details on a climatological conditions during times of peak demand or output; while a cold season month can highlight a minimum baseline for operating conditions across the energy sector.

To analyze a specific month, we'll retrieve some of the absolute AMY data of interest shown above. Please select in the explore panel above 'Absolute' as your AMY Type, and either 'Historical' or 'Warming Level Future' with a warming level of your choosing. A progress bar is provided when running the cell below, as this step may take a little while. 

In [ ]:
my_data = client.compute(app.retrieve().mean(dim="simulation").isel(scenario=0))
progress(my_data, multi=False)

In [ ]:
my_data = my_data.result()

In [ ]:
from climakitae.tmy import tmy_calc
amy_data = tmy_calc(my_data)

Grab a particular month from the Average Meteorological Year. Here, we select September.

In [ ]:
month = 'Sep'
one_month = amy_data.iloc[amy_month(month)[0]:amy_month(month)[1]]
one_month = pd.DataFrame(one_month.values.ravel(), columns=['Hour'])
one_month.name = my_data.name + ' (' + my_data.attrs['units'] + ')'

Next, we will visualize what one month in the Average Meteorological Year looks like for our selected month.

In [ ]:
one_month.hvplot.line(x='Hour of the Month',
                      ylabel=one_month.name,
                      title='Average Meteorological Year: ' + app.selections.location.cached_area
                      + '\n' + datetime.datetime.strptime(month, '%b').date().strftime('%B') + ' ' + app.selections.scenario[0], # want to grab computation option instead here (+ warming level of choice if future)
                      height=400, width=650)

## Step 3: Export data

To export the Average Meteorologial Year data, use the following code:

In [ ]:
amy_data.to_csv('my_filename' + '.csv')

To export the data that is used to calculate the Average Meteorological Year (not the AMY result), first pick a format from the dropdown menu.

- We recommend NetCDF, which will work with any number of variables and dimensions in your dataset
- CSV works best up to 2-dimensional data (e.g., lon x lat), and will be compressed and exported with a separate metadata file
- GeoTIFF is not possible for Average Meteorological Year as the time series data does not retain a spatial component

In [ ]:
app.export_as()

Next, write in the object you wish to export and your desired filename (in single or double quotation marks).

In [ ]:
app.export_dataset(my_data, 'my_filename')

Lastly, when you are done, close your cluster resources to free them up for the next time you work. 

In [ ]:
client.close()